## Problem Description

The aim of this problem is to figure out the optimal locations and treatment capacity of COVID-19 healthcare facilities by minimizing the cost of opening these temporary treatment facilities, as well as the total cost of patients driving from a county to the healthcare facility.

This problem considers 9 fictional counties in California, which are already equipped with COVID-19 treatment facilities and have an option to build additional temporary facilities at specified locations. The following details are provided:

1. It costs $500,000 to construct one temporary facility that can treat 100 patients

2. It costs $5 extra in driving costs for an increase of 10 miles in distance to a treatment facility

3. Existing facilities have a capacity of 80% of the forecasted demand, except for county 9, which is assumed to have excess capacity

The distance between each county, and each facility may be computed using their coordinates(which are in tens of miles).

**Reference:**

*This problem is taken from Gurobi’s resources (https://www.gurobi.com/resource/covid-19-healthcare-facility-capacity-optimization/) which is based on a published paper (Katherine Klise and Michael Bynum. Facility Location Optimization Model for COVID-19 Resources. April 2020. Joint DOE Laboratory Pandemic Modeling and Analysis Capability. SAND2020-4693R.).*

*This problem was solved as part of MILP Assignment in the course Mathematical Optimization with GAMS and Pyomo (Python) on udemy. (https://www.udemy.com/course/mathematical-optimization-with-gams-and-pyomo-python/)*

## Data

|County|Coordinates (centroid)|Forecasted demand|
| :- | :--: | :--: |
|County 1|[1,1.5]|351|
|County 2|[3,1]|230|
|County 3|[5.5,1.5]|529|
|County 4|[1,4.5]|339|
|County 5|[3,3.5]|360|
|County 6|[5.5,4.5]|527|
|County 7|[1,8]|469|
|County 8|[3,6]|234|
|County 9|[4.5,8]|500|

<br>

|Existing Facility|Coordinates|Capacity|
| :- | :--: | :--: |
|Facility 1|[1,2]|281|
|Facility 2|[2.5,1]|187|
|Facility 3|[5,1]|200|
|Facility 4|[6.5,3.5]|223|
|Facility 5|[1,5]|281|
|Facility 6|[3,4]|281|
|Facility 7|[5,4]|222|
|Facility 8|[6.5,5.5]|200|
|Facility 9|[1,8.5]|250|
|Facility 10|[1.5,9.5]|125|
|Facility 11|[8.5,6]|187|
|Facility 12|[5,8]|300|
|Facility 13|[3,9]|300|
|Facility 14|[6,9]|243|

<br>

|Temporary Facility|Coordinates|Capacity|
| :- | :--: | :--: |
|Facility 1|[1.5,1]|100|
|Facility 2|[3.5,1.5]|100|
|Facility 3|[5.5,2.5]|100|
|Facility 4|[1.5,3.5]|100|
|Facility 5|[3.5,2.5]|100|
|Facility 6|[4.5,4.5]|100|
|Facility 7|[1.5,6.5]|100|
|Facility 8|[3.5,6.5]|100|
|Facility 9|[5.5,6.5]|100|

## Solution

**Sets and Indices:**

$c \in C$: Index and set of counties.

$f \in F$: Index and set of existing facilities.

$t \in T$: Index and set of temporary facilities.

**Variables:**

$x_{t} \in \{0,1\}$: $x$ is 1 for temporary facility $t$ if its built else 0.

$a_{c,f} \in \mathbb{I}^+$: No. of people from county $c$ treated at existing facility $f$.

$b_{c,t} \in \mathbb{I}^+$: No. of people from county $c$ treated at temporary facility $t$.

$z_{t} \in \mathbb{I}^+$: Extra capacity needed at temporary facility $t$.

**Parameters:**

$Demand_{c} \in \mathbb{I}^+$: Expected no. of patients from county $c$.

$Ext\_Cap_{f} \in \mathbb{I}^+$: Capacity at existing facility $f$.

$Temp\_Cap_{t} \in \mathbb{I}^+$: Capacity at temporary facility $t$.

$Ext\_Dist_{c,f} \in \mathbb{R}^+$: Distance from county $c$ to existing facility $f$.

$\text{Temp_Dist}_{c,t} \in \mathbb{I}^+$: Distance from county $c$ to temporary facility $t$.

$Temp\_Cost = 500000$: Cost of setting up a temporary facility.

$Travel\_Cost = 5$: Cost of travelling 10 miles.

$bigM = 5*Temp\_Cost$: Big-M parameter to penalize extra capacity at a temporary facility.

**Objective Function:**

\begin{equation}
\text{Min} \sum_{c \in C} \sum_{f \in F} Travel\_Cost * a_{c,f} * Ext\_Dist_{c,f} + \\ \sum_{c \in C} \sum_{t \in T} Travel\_Cost * b_{c,t} * Temp\_Dist_{c,t} + \\ \sum_{t \in T} Temp\_Cost*x_{t} + bigM*z_{t}
\tag{0}
\end{equation}

**Constraints:**

\begin{equation}
\sum_{f \in F} a_{c,f} + \sum_{t \in T} b_{c,t} = Demand_{c} \quad \forall c \in C
\tag{1}
\end{equation}

\begin{equation}
\sum_{c \in C} a_{c,f} \le Ext\_Cap_{f} \quad \forall f \in F
\tag{2}
\end{equation}

\begin{equation}
\sum_{c \in C} b_{c,t} \le x_{t}*Temp\_Cap_{t}+z_{t} \quad \forall t \in T
\tag{3}
\end{equation}

In [1]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyomo

!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
import os
import numpy as np

     |████████████████████████████████| 9.2 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [2]:
# Model
model = pyomo.ConcreteModel()

# Set
model.county = pyomo.RangeSet(9)
model.facility = pyomo.RangeSet(14)
model.temp_facility = pyomo.RangeSet(9)

# Variables
model.temp_facility_x = pyomo.Var(model.temp_facility, domain=pyomo.Binary)
model.county_facility_var = pyomo.Var(model.county, model.facility, domain=pyomo.NonNegativeIntegers)
model.county_temp_facility_var = pyomo.Var(model.county, model.temp_facility, domain=pyomo.NonNegativeIntegers)
model.extra_capacity = pyomo.Var(model.temp_facility, domain=pyomo.NonNegativeIntegers)

# Parameters
## 1. County Demands
county_demand_dict = {1:351, 2:230, 3:529, 4:339, 5:360, 6:527, 7:469, 8:234, 9:500}
model.county_demand = pyomo.Param(model.county, initialize = county_demand_dict)

## 2. Facility capacity
facility_capacity_dict = {1:281, 2:187, 3:200, 4:223, 5:281, 6:281, 7:222, 8:200, 9:250, 10:125, 11:187, 12:300, 13:300, 14:243}
model.facility_capacity = pyomo.Param(model.facility, initialize = facility_capacity_dict)

## 3. Temp Facility capacity
model.temp_facility_capacity = pyomo.Param(model.temp_facility, initialize = {1:100, 2:100, 3:100, 4:100, 5:100, 6:100, 7:100, 8:100, 9:100})

## 4. Coordinates of County, Facility and Temp Facility
model.county_coord = pyomo.Param(model.county, initialize = {1:{'x':1,'y':1.5}, 2:{'x':3,'y':1}, 3:{'x':5.5,'y':1.5},
                                                             4:{'x':1,'y':4.5}, 5:{'x':3,'y':3.5}, 6:{'x':5.5,'y':4.5},
                                                             7:{'x':1,'y':8}, 8:{'x':3,'y':6}, 9:{'x':4.5,'y':8}})

model.facility_coord = pyomo.Param(model.facility, initialize = {1:{'x':1,'y':2}, 2:{'x':2.5,'y':1}, 3:{'x':5,'y':1}, 4:{'x':6.5,'y':3.5},5:{'x':1,'y':5},
                                                                 6:{'x':3,'y':4}, 7:{'x':5,'y':4}, 8:{'x':6.5,'y':5.5},9:{'x':1,'y':8.5}, 10:{'x':1.5,'y':9.5},
                                                                 11:{'x':8.5,'y':6}, 12:{'x':5,'y':8}, 13:{'x':3,'y':9}, 14:{'x':6,'y':9}})

model.temp_facility_coord = pyomo.Param(model.temp_facility, initialize = {1:{'x':1.5,'y':1}, 2:{'x':3.5,'y':1.5}, 3:{'x':5.5,'y':2.5},
                                                                           4:{'x':1.5,'y':3.5}, 5:{'x':3.5,'y':2.5}, 6:{'x':4.5,'y':4.5},
                                                                           7:{'x':1.5,'y':6.5}, 8:{'x':3.5,'y':6.5}, 9:{'x':5.5,'y':6.5}})

## 5. Distances between County-Facility and County-TempFacility
def distance1(model, c, f):
  return ( (model.county_coord[c]['x'] - model.facility_coord[f]['x'])**2 + (model.county_coord[c]['y'] - model.facility_coord[f]['y'])**2 )**(0.5)
model.county_facility_dist = pyomo.Param(model.county, model.facility, initialize = distance1)

def distance2(model, c, f):
  return ( (model.county_coord[c]['x'] - model.temp_facility_coord[f]['x'])**2 + (model.county_coord[c]['y'] - model.temp_facility_coord[f]['y'])**2 )**(0.5)
model.county_temp_facility_dist = pyomo.Param(model.county, model.temp_facility, initialize = distance2)

## 6. Cost of building a Temp Facility
model.cost_temp_facility = pyomo.Param(initialize = 500000)

## 7. Cost of driving for 10miles
model.cost_travel = pyomo.Param(initialize = 5)

## 8. Big M for penalizing extra capacity
model.M = pyomo.Param(initialize = 5*model.cost_temp_facility)

# Constraints
def rule1(model, c):
  return sum(model.county_facility_var[c,f] for f in model.facility) + sum(model.county_temp_facility_var[c,f] for f in model.temp_facility) == model.county_demand[c]
model.equ1 = pyomo.Constraint(model.county, rule = rule1)

def rule2(model, f):
  return sum(model.county_facility_var[c,f] for c in model.county) <= model.facility_capacity[f]
model.equ2 = pyomo.Constraint(model.facility, rule = rule2)

def rule3(model, f):
  return sum(model.county_temp_facility_var[c,f] for c in model.county) <= model.temp_facility_x[f]*model.temp_facility_capacity[f]+model.extra_capacity[f]
model.equ3 = pyomo.Constraint(model.temp_facility, rule = rule3)

# Objective
def obj_fnc(model):
  value = sum(sum(model.cost_travel*model.county_facility_dist[c,f]*model.county_facility_var[c,f] for c in model.county) 
                for f in model.facility) + \
          sum(sum(model.cost_travel*model.county_temp_facility_dist[c,f]*model.county_temp_facility_var[c,f] for c in model.county) 
                for f in model.temp_facility) + \
          sum((model.cost_temp_facility*model.temp_facility_x[f] + model.M*model.extra_capacity[f]) for f in model.temp_facility)
  return value
model.obj = pyomo.Objective(rule = obj_fnc
                            ,
                            sense = pyomo.minimize)

# Solver options
results = pyomo.SolverFactory('glpk',executable='/usr/bin/glpsol').solve(model)

results.write()

print('\n RESULTS \n')

print('Optimal cost for setting up facilities is $',model.obj(),'\n')
for f in model.temp_facility:
  if model.temp_facility_x[f]():
    print('Temp Facility',f,'should be constructed')

print('\n Report on Facilities \n')
for c in model.county:
  for f in model.facility:
      if model.county_facility_var[c,f]() != 0:
        print(model.county_facility_var[c,f](),'people from County',c,'go to Existing Facility',f)

print('\n Report on Temp Facilities \n')
for c in model.county:
  for f in model.temp_facility:
    if model.temp_facility_x[f]():
      if model.county_temp_facility_var[c,f]() != 0:
        print(model.county_temp_facility_var[c,f](),'people from County',c,'go to Temp Facility',f)

    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (county_coord) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0) (called from
    /usr/local/lib/python3.7/dist-
    packages/pyomo/core/base/indexed_component.py:623)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (facility_coord) to be 'Any', you can
    suppress this warning by explicitly specifying 'within=Any' to the Param
    constructor.  (deprecated in 5.6.9, will be removed in 6.0) (called from
    /usr/local/lib/python3.7/dist-
    packages/pyomo/core/base/indexed_component.py:623)
    we will be changing that default to 'Reals' in the future.  If you really
    intend the domain of this Param (temp_facility_coord) to be 'Any', you can
    suppress this warning by explicitly specifying 'w